In [1]:
# Importez les bibliothèques nécessaires
!pip install flask flask-swagger-ui flask-restx pandas scikit-learn numpy joblib psycopg2-binary sqlalchemy

# Code complet de l'application Flask avec intégration Swagger
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
import joblib
import os
import warnings
warnings.filterwarnings('ignore')

# Importer les modules Flask et Swagger
from flask import Flask, request, jsonify
from flask_restx import Api, Resource, fields

# Création du dossier pour stocker les modèles
os.makedirs('models', exist_ok=True)

# Paramètres pour la connexion à PostgreSQL
host = 'localhost'
port = '5432'
database = 'Sus_DW'
user = 'postgres'
password = '12345678'

# Création du moteur SQLAlchemy
engine = create_engine(f'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}')

print("Chargement des données...")
# Connexion et lecture des tables
equipements_df = pd.read_sql("SELECT * FROM dim_equipements", engine)
notifications_df = pd.read_sql("SELECT * FROM dim_notifications", engine)

# Nettoyage initial
equipements_df = equipements_df.dropna(subset=['Equipment_ID'])
notifications_df = notifications_df.dropna(subset=['Equipement_id', 'Priorite'])

# Création du label binaire
notif_critiques = notifications_df[notifications_df['Priorite'] == 'Critique'].copy()
notif_critiques.loc[:, 'label'] = 1
labels = notif_critiques.groupby('Equipement_id')['label'].max().reset_index()

# Fusion des données
dataset = pd.merge(equipements_df, labels, left_on='Equipment_ID', right_on='Equipement_id', how='left')
dataset['label'] = dataset['label'].fillna(0)
dataset = dataset.drop(columns=['Equipement_id'])

# Colonnes sélectionnées
features = ['Manufacturer', 'Energy_Type', 'Estimated_Lifetime_Years', 'CO2_Emissions_kg', 'Energy_Consumption_kWh']
df_model = dataset[features + ['label']].copy()

# Encodage des variables catégorielles
label_encoders = {}
for col in ['Manufacturer', 'Energy_Type']:
    df_model[col] = df_model[col].astype(str)
    le = LabelEncoder()
    df_model[col] = le.fit_transform(df_model[col])
    label_encoders[col] = le

# Conversion des colonnes numériques
for col in ['Estimated_Lifetime_Years', 'CO2_Emissions_kg', 'Energy_Consumption_kWh']:
    df_model[col] = pd.to_numeric(df_model[col], errors='coerce')

# Séparer features et cible
X = df_model.drop(columns=['label'])
y = df_model['label']

# Imputation complète (numériques + encodées)
X = X.dropna(axis=1, how='all')
imputer = SimpleImputer(strategy='median')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# Split train/test
X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=0.3, random_state=42)

# Standardisation
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Entraînement des modèles...")
# Entraînement du modèle KNN
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)
print("✅ Modèle KNN entraîné")

# Entraînement du modèle SVM
svm = SVC(kernel='rbf', C=1, gamma='scale', probability=True)
svm.fit(X_train_scaled, y_train)
print("✅ Modèle SVM entraîné")

# Entraînement du modèle Decision Tree
dtree = DecisionTreeClassifier(max_depth=5, random_state=42)
dtree.fit(X_train, y_train)  # pas besoin de scaling
print("✅ Modèle Decision Tree entraîné")

# Sauvegarde des préprocesseurs et des modèles
joblib.dump(label_encoders, 'models/label_encoders.pkl')
joblib.dump(imputer, 'models/imputer.pkl')
joblib.dump(scaler, 'models/scaler.pkl')
joblib.dump(knn, 'models/knn_model.pkl')
joblib.dump(svm, 'models/svm_model.pkl')
joblib.dump(dtree, 'models/dtree_model.pkl')
joblib.dump(list(X.columns), 'models/feature_names.pkl')
print("✅ Tous les modèles et préprocesseurs ont été sauvegardés")

# Définition de la fonction de prétraitement
def preprocess_input(data):
    """Prétraite les données d'entrée pour la prédiction"""
    # Créer un DataFrame avec les données d'entrée
    input_df = pd.DataFrame([data])
    
    # Vérifier si toutes les colonnes requises sont présentes
    feature_names = list(X.columns)
    for col in feature_names:
        if col not in input_df.columns:
            input_df[col] = np.nan
    
    # Sélectionner uniquement les colonnes utilisées par les modèles
    input_df = input_df[feature_names]
    
    # Encoder les variables catégorielles
    for col, encoder in label_encoders.items():
        if col in input_df.columns:
            input_df[col] = input_df[col].astype(str)
            # Gérer les catégories inconnues
            input_df[col] = input_df[col].apply(lambda x: x if x in encoder.classes_ else encoder.classes_[0])
            input_df[col] = encoder.transform(input_df[col])
    
    # Convertir les colonnes numériques
    for col in ['Estimated_Lifetime_Years', 'CO2_Emissions_kg', 'Energy_Consumption_kWh']:
        if col in input_df.columns:
            input_df[col] = pd.to_numeric(input_df[col], errors='coerce')
    
    # Appliquer l'imputation
    input_imputed = pd.DataFrame(imputer.transform(input_df), columns=input_df.columns)
    
    return input_imputed

# Création de l'application Flask avec flask-restx pour Swagger
app = Flask(__name__)
api = Api(app, version='1.0', title='API de Prédiction pour Équipements',
          description='API pour prédire les problèmes critiques des équipements avec 3 modèles ML')

# Créer un namespace pour les endpoints de prédiction
ns = api.namespace('predict', description='API de prédiction')

# Définir le modèle de données d'entrée pour Swagger
input_model = api.model('EquipementInput', {
    'Manufacturer': fields.String(required=True, description='Fabricant de l\'équipement', example='ABB'),
    'Energy_Type': fields.String(required=True, description='Type d\'énergie', example='Électrique'),
    'Estimated_Lifetime_Years': fields.Float(required=True, description='Durée de vie estimée (années)', example=15),
    'CO2_Emissions_kg': fields.Float(required=True, description='Émissions de CO2 (kg)', example=120),
    'Energy_Consumption_kWh': fields.Float(required=True, description='Consommation d\'énergie (kWh)', example=1500)
})

# Définir les modèles de données de sortie pour Swagger
prediction_output = api.model('PredictionOutput', {
    'model': fields.String(description='Nom du modèle utilisé'),
    'prediction': fields.Integer(description='Prédiction (0: pas de problème, 1: problème critique)'),
    'probability': fields.Float(description='Probabilité de problème critique'),
    'prediction_text': fields.String(description='Description textuelle de la prédiction')
})

single_model_output = api.model('SingleModelOutput', {
    'prediction': fields.Integer(description='Prédiction (0: pas de problème, 1: problème critique)'),
    'probability': fields.Float(description='Probabilité de problème critique')
})

all_models_output = api.model('AllModelsOutput', {
    'knn': fields.Nested(single_model_output),
    'svm': fields.Nested(single_model_output),
    'dtree': fields.Nested(single_model_output),
    'ensemble': fields.Nested(prediction_output)
})

@ns.route('/knn')
class KNNPrediction(Resource):
    @ns.expect(input_model)
    @ns.marshal_with(prediction_output, code=200, description='Prédiction réussie')
    @ns.response(400, 'Données d\'entrée invalides')
    @ns.response(500, 'Erreur de traitement')
    def post(self):
        """Prédiction avec le modèle KNN"""
        data = request.json
        if not data:
            api.abort(400, 'Aucune donnée fournie')
        
        try:
            # Prétraiter les données
            input_imputed = preprocess_input(data)
            
            # Standardiser les données
            input_scaled = scaler.transform(input_imputed)
            
            # Faire la prédiction
            prediction = knn.predict(input_scaled)[0]
            probability = knn.predict_proba(input_scaled)[0][1]
            
            return {
                'model': 'KNN',
                'prediction': int(prediction),
                'probability': float(probability),
                'prediction_text': 'Problème critique probable' if prediction == 1 else 'Pas de problème critique'
            }
        
        except Exception as e:
            api.abort(500, str(e))

@ns.route('/svm')
class SVMPrediction(Resource):
    @ns.expect(input_model)
    @ns.marshal_with(prediction_output, code=200, description='Prédiction réussie')
    @ns.response(400, 'Données d\'entrée invalides')
    @ns.response(500, 'Erreur de traitement')
    def post(self):
        """Prédiction avec le modèle SVM"""
        data = request.json
        if not data:
            api.abort(400, 'Aucune donnée fournie')
        
        try:
            # Prétraiter les données
            input_imputed = preprocess_input(data)
            
            # Standardiser les données
            input_scaled = scaler.transform(input_imputed)
            
            # Faire la prédiction
            prediction = svm.predict(input_scaled)[0]
            probability = svm.predict_proba(input_scaled)[0][1]
            
            return {
                'model': 'SVM',
                'prediction': int(prediction),
                'probability': float(probability),
                'prediction_text': 'Problème critique probable' if prediction == 1 else 'Pas de problème critique'
            }
        
        except Exception as e:
            api.abort(500, str(e))

@ns.route('/dtree')
class DecisionTreePrediction(Resource):
    @ns.expect(input_model)
    @ns.marshal_with(prediction_output, code=200, description='Prédiction réussie')
    @ns.response(400, 'Données d\'entrée invalides')
    @ns.response(500, 'Erreur de traitement')
    def post(self):
        """Prédiction avec le modèle Decision Tree"""
        data = request.json
        if not data:
            api.abort(400, 'Aucune donnée fournie')
        
        try:
            # Prétraiter les données
            input_imputed = preprocess_input(data)
            
            # Pas besoin de standardiser pour Decision Tree
            
            # Faire la prédiction
            prediction = dtree.predict(input_imputed)[0]
            probability = dtree.predict_proba(input_imputed)[0][1]
            
            return {
                'model': 'Decision Tree',
                'prediction': int(prediction),
                'probability': float(probability),
                'prediction_text': 'Problème critique probable' if prediction == 1 else 'Pas de problème critique'
            }
        
        except Exception as e:
            api.abort(500, str(e))

@ns.route('/all')
class EnsemblePrediction(Resource):
    @ns.expect(input_model)
    @ns.marshal_with(all_models_output, code=200, description='Prédiction réussie')
    @ns.response(400, 'Données d\'entrée invalides')
    @ns.response(500, 'Erreur de traitement')
    def post(self):
        """Prédiction avec tous les modèles et vote majoritaire"""
        data = request.json
        if not data:
            api.abort(400, 'Aucune donnée fournie')
        
        try:
            # Prétraiter les données
            input_imputed = preprocess_input(data)
            input_scaled = scaler.transform(input_imputed)
            
            # Faire les prédictions avec chaque modèle
            knn_pred = knn.predict(input_scaled)[0]
            knn_prob = knn.predict_proba(input_scaled)[0][1]
            
            svm_pred = svm.predict(input_scaled)[0]
            svm_prob = svm.predict_proba(input_scaled)[0][1]
            
            dtree_pred = dtree.predict(input_imputed)[0]
            dtree_prob = dtree.predict_proba(input_imputed)[0][1]
            
            # Vote majoritaire
            ensemble_pred = 1 if (knn_pred + svm_pred + dtree_pred) >= 2 else 0
            ensemble_prob = (knn_prob + svm_prob + dtree_prob) / 3
            
            return {
                'knn': {
                    'prediction': int(knn_pred),
                    'probability': float(knn_prob)
                },
                'svm': {
                    'prediction': int(svm_pred),
                    'probability': float(svm_prob)
                },
                'dtree': {
                    'prediction': int(dtree_pred),
                    'probability': float(dtree_prob)
                },
                'ensemble': {
                    'prediction': int(ensemble_pred),
                    'probability': float(ensemble_prob),
                    'prediction_text': 'Problème critique probable' if ensemble_pred == 1 else 'Pas de problème critique'
                }
            }
        
        except Exception as e:
            api.abort(500, str(e))

# Pour démarrer le serveur dans Jupyter
from IPython.display import display, HTML
from threading import Thread
import webbrowser

# Lancer le serveur Flask dans un thread
def run_flask():
    app.run(debug=False, host='0.0.0.0', port=5000, use_reloader=False)

flask_thread = Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

# Message d'information sur l'utilisation de Swagger UI
swagger_info = """
<div style="background-color: #f0f0f0; padding: 15px; border-radius: 5px; margin: 10px 0;">
<h3>🚀 API avec Swagger UI démarrée!</h3>
<p>L'API est maintenant accessible à l'adresse: <b>http://localhost:5000</b></p>
<p>La documentation Swagger UI est disponible à: <b>http://localhost:5000/</b></p>
<p>Avec Swagger UI, vous pouvez:</p>
<ul>
  <li>Explorer la documentation de l'API</li>
  <li>Tester chaque endpoint directement depuis l'interface</li>
  <li>Voir les modèles de données d'entrée et de sortie</li>
</ul>
<p>Instructions d'utilisation:</p>
<ol>
  <li>Cliquez sur un endpoint (par exemple <code>/predict/all</code>)</li>
  <li>Cliquez sur le bouton "Try it out"</li>
  <li>Modifiez les données d'entrée si nécessaire</li>
  <li>Cliquez sur "Execute" pour envoyer la requête</li>
  <li>Les résultats apparaîtront sous la section "Responses"</li>
</ol>
</div>
"""

# Ouvrir automatiquement le navigateur
def open_browser():
    import time
    time.sleep(1)
    webbrowser.open('http://localhost:5000')

browser_thread = Thread(target=open_browser)
browser_thread.daemon = True
browser_thread.start()

display(HTML(swagger_info))

Chargement des données...
Entraînement des modèles...
✅ Modèle KNN entraîné
✅ Modèle SVM entraîné
✅ Modèle Decision Tree entraîné
✅ Tous les modèles et préprocesseurs ont été sauvegardés


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.1.15:5000
Press CTRL+C to quit
127.0.0.1 - - [28/Apr/2025 21:10:19] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2025 21:10:20] "GET /swaggerui/droid-sans.css HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2025 21:10:20] "GET /swaggerui/swagger-ui.css HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2025 21:10:20] "GET /swaggerui/swagger-ui-bundle.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2025 21:10:20] "GET /swaggerui/swagger-ui-standalone-preset.js HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2025 21:10:20] "GET /swagger.json HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2025 21:10:20] "GET /swaggerui/favicon-32x32.png HTTP/1.1" 200 -
127.0.0.1 - - [28/Apr/2025 21:10:57] "POST /predict/knn HTTP/1.1" 200 -
